<a href="https://colab.research.google.com/github/StefaneBG/Flora-projetoFinal/blob/main/Flora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import auth
from google.cloud import storage
from google.colab import files
from google.cloud import bigquery
import inspect

#Funções

##Função Enviar para Bucket

In [ ]:
def enviar_p_bucket(dataframe,destination_folder):

    project_id = ''
    bucket_name=''

    try:
        # Determinar o nome da variável associada ao DataFrame
        caller_locals = inspect.currentframe().f_back.f_locals
        dataframe_name = [name for name, obj in caller_locals.items() if obj is dataframe]
        if dataframe_name:
            dataframe_name = dataframe_name[0]
        else:
            dataframe_name = "dataframe"

        # Nome do arquivo local
        local_file_name = f"{dataframe_name}-{destination_folder}.csv"

        # Caminho no bucket (diretório virtual)
        destination_blob_name = f"{destination_folder}/{local_file_name}"

        # Autenticar no Colab
        auth.authenticate_user()

        # Configurar o projeto no Google Cloud
        !gcloud config set project {project_id}

        # Salvar o DataFrame localmente como CSV
        dataframe.to_csv(local_file_name, index=False)

        # Inicializar o cliente do Google Cloud Storage
        client = storage.Client()
        bucket = client.bucket(bucket_name)

        # Enviar arquivo ao bucket
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(local_file_name)

        return f"Arquivo '{local_file_name}' carregado com sucesso em '{destination_blob_name}' no bucket '{bucket_name}'."

    except Exception as e:
        return f"Erro ao realizar o upload: {e}"

##Função Enviar para BigQuery

In [ ]:
def enviar_para_bigquery(dataframe, project_id='', dataset_id=''):

    try:
      # Determinar o nome da variável associada ao DataFrame
        caller_locals = inspect.currentframe().f_back.f_locals
        dataframe_name = [name for name, obj in caller_locals.items() if obj is dataframe]
        if dataframe_name:
            dataframe_name = dataframe_name[0]
        else:
            dataframe_name = "dataframe"
        # Nome do arquivo local
        table_name = f"{dataframe_name}"

        # Autenticar no Google Cloud
        auth.authenticate_user()

        # Criar o cliente do BigQuery
        client = bigquery.Client()

        # Construir o ID completo da tabela
        table_id = f"{project_id}.{dataset_id}.{table_name}"

        # Enviar os dados para o BigQuery
        dataframe.to_gbq(table_id, project_id=project_id, if_exists='replace')

        return f"Dados enviados com sucesso para {table_id}."

    except Exception as e:
        return f"Erro ao enviar dados para o BigQuery: {e}"

# Carregamento dos Dados

In [ ]:
notaVenda = pd.read_csv('',header=0)
notaVenda.head()

In [ ]:
coordenador = pd.read_csv('',header=0)
coordenador.head()

In [ ]:
cliente = pd.read_csv('',header=0)
cliente.head()

In [ ]:
divisaoComercial = pd.read_csv('',header=0)
divisaoComercial.head()

In [ ]:
produto = pd.read_csv('',header=0)
produto.head()

In [ ]:
empresa = pd.read_csv('',header=0)
empresa.head()

In [ ]:
mixIdeal = pd.read_csv('',header=0)
mixIdeal.head()

## Informações dos Dados

In [ ]:
notaVenda.info()

In [ ]:
coordenador.info()

In [ ]:
cliente.info()

In [ ]:
divisaoComercial.info()

In [ ]:
produto.info()

In [ ]:
empresa.info()

In [ ]:
mixIdeal.info()

#Enviar dados não tratados para Bucket

In [ ]:
list_df = [mixIdeal, empresa, produto, divisaoComercial, cliente, coordenador, notaVenda]
for list in list_df:
  enviar_p_bucket(list, "dados-nao-tratados")


# Tratamento dos Dados

In [ ]:
# Converter as datas para datetime
notaVenda['Data Faturamento'] = pd.to_datetime(notaVenda['Data Faturamento'])
meses_portugues = {
    'jan': 'Jan', 'fev': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'mai': 'May', 'jun': 'Jun',
    'jul': 'Jul', 'ago': 'Aug', 'set': 'Sep', 'out': 'Oct', 'nov': 'Nov', 'dez': 'Dec'
    }
notaVenda['Mês'] = notaVenda['Mês'].str[:3].map(meses_portugues) + notaVenda['Mês'].str[3:]
notaVenda['Mês'] = pd.to_datetime(notaVenda['Mês'], format='%b %Y')
notaVenda = notaVenda.drop(columns=['Ano'])

# Converter colunas para string
notaVenda['nCdEmpresa'] = notaVenda['nCdEmpresa'].astype(str)
notaVenda['nCdProduto'] = notaVenda['nCdProduto'].astype(str)
notaVenda['nCdDivisaoComercial'] = notaVenda['nCdDivisaoComercial'].astype(str)
notaVenda['nCdTerceiroCoordenador'] = notaVenda['nCdTerceiroCoordenador'].astype(str)
notaVenda['nCdTerceiroCentralizador'] = notaVenda['nCdTerceiroCentralizador'].astype(str)

# Converter 'Fat Cxs' e 'Fat R$' para float
notaVenda['Fat Cxs'] = notaVenda['Fat Cxs'].str.replace(',', '.').astype(float)
notaVenda['Fat R$'] = notaVenda['Fat R$'].str.replace(',', '.').astype(float)
notaVenda['Quantidade de Caixas'] = notaVenda['Fat R$'] / notaVenda['Fat Cxs']

# Renomear colunas
notaVenda.rename(columns={'Mês': 'Mes', 'Fat R$': 'FatValor'}, inplace=True)
notaVenda.info()

In [ ]:
cliente['nCdTerceiroCentralizador'] = cliente['nCdTerceiroCentralizador'].astype(str)
cliente.info()

In [ ]:
divisaoComercial['nCdDivisaoComercial'] = divisaoComercial['nCdDivisaoComercial'].astype(str)
divisaoComercial['DivisaoComercial'] = divisaoComercial['DivisaoComercial'].astype(str)
divisaoComercial.info()

In [ ]:
produto['nCdProduto'] = produto['nCdProduto'].astype(str)
produto['Produto'] = produto['Produto'].astype(str)
produto.info()

In [ ]:
empresa['nCdEmpresa'] = empresa['nCdEmpresa'].astype(str)
empresa['Empresa'] = empresa['Empresa'].astype(str)
empresa.info()

In [ ]:
mixIdeal['ncdProduto '] = mixIdeal['ncdProduto '].astype(str)
mixIdeal.rename(columns={'ncdProduto ': 'nCdProduto', 'Produto ': 'Produto'}, inplace=True)
mixIdeal.info()

In [ ]:
coordenador['nCdTerceiroCoordenador'].unique()

In [ ]:
coordenador['nCdTerceiroCoordenador'] = coordenador['nCdTerceiroCoordenador'].replace("-", "Sem Coordenador Identificado")
coordenador['nCdTerceiroCoordenador'].unique()

In [ ]:
notaVenda['nCdTerceiroCoordenador'].unique()

In [ ]:
notaVenda['nCdTerceiroCoordenador'] = notaVenda['nCdTerceiroCoordenador'].replace("-", "Sem Coordenador Identificado")
notaVenda['nCdTerceiroCoordenador'].unique()

## Duplicadas

In [ ]:
def procurar_duplicadas(dataframe):
  duplicadas = dataframe[dataframe.duplicated(keep=False)]
  return duplicadas
mixIdeal['nCdProduto'].value_counts()

In [ ]:
duplicated_rows = mixIdeal[mixIdeal['nCdProduto'].duplicated(keep=False)].sort_values(by=['nCdProduto'])
display(duplicated_rows)

##Excluindo linhas duplicadas

In [ ]:
mixIdeal = mixIdeal.drop_duplicates(subset=['nCdProduto'])
mixIdeal['nCdProduto'].value_counts()

#Enviar dados tratados para Bucket

In [ ]:
list_df = [mixIdeal, empresa, produto, divisaoComercial, cliente, coordenador, notaVenda]
for list in list_df:
  enviar_p_bucket(list, "dados-tratados")

#Perguntas e Respostas

In [ ]:
notaVenda['FatValor'] = notaVenda['FatValor'] / 1000000
notaVenda['Fat Cxs'] = notaVenda['Fat Cxs'] / 1000000
notaVenda['Quantidade de Caixas'] = notaVenda['Quantidade de Caixas'] / 1000000

##(INSERIDO)1.Clientes que compraram produtos do mixIdeal e suas quantidades por produto


In [ ]:
print(f"Quantidade de produtos do mix ideal é: {len(mixIdeal)}")
# Clientes que compraram produtos do mixIdeal e suas quantidades por produto
# Agrupar por 'nCdTerceiroCentralizador' e 'nCdProduto', contando as ocorrências e somando 'Quantidade de Caixas'
clientes_qtds_mixIdeal = notaVenda.groupby(['nCdTerceiroCentralizador', 'nCdProduto']).agg(
    qtds_produtos_comprados_mixIdeal=('nCdProduto', 'size'),
    quantidade_caixas=('Quantidade de Caixas', 'sum')
).reset_index().sort_values(by='qtds_produtos_comprados_mixIdeal', ascending=False)

# Exibir o resultado
display(clientes_qtds_mixIdeal)

##(INSERIDO)2.Quais clientes (terceiros centralizadores) estão comprando o mix ideal?

In [ ]:
# Passo 1: Obter a lista completa de clientes (terceiros centralizadores)
clientes_totais = notaVenda['nCdTerceiroCentralizador'].unique()

# Passo 2: Criar um DataFrame para os clientes que compraram do mixIdeal
dfComMixIdeal = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='inner')

# Criar o DataFrame 'mix' com os produtos únicos e combinação do mix
mix = dfComMixIdeal[['nCdTerceiroCentralizador', 'nCdProduto']].groupby('nCdTerceiroCentralizador')['nCdProduto'].unique().reset_index(name='produtos_unicos')
mix['cobertura_mix_clientes'] = (mix['produtos_unicos'].apply(len) / len(mixIdeal['nCdProduto'])) * 100
mix['SKUs_Comprados_clientes'] = mix['produtos_unicos'].apply(len)

# Total de produtos do mixIdeal comprados por cliente
clientes_com_mix = dfComMixIdeal.groupby('nCdTerceiroCentralizador')['nCdProduto'].count().reset_index(name='total_compra_mix_cliente')

# Unir os dados para incluir a cobertura do mix, os produtos únicos comprados e a quantidade
status_cliente_mix = pd.merge(
    clientes_com_mix,
    mix[['nCdTerceiroCentralizador', 'cobertura_mix_clientes', 'produtos_unicos', 'SKUs_Comprados_clientes']],
    on='nCdTerceiroCentralizador',
    how='left'
)

# Passo 3: Adicionar os clientes que não compraram nenhum produto do mixIdeal
clientes_sem_mix = pd.DataFrame({
    'nCdTerceiroCentralizador': [cliente for cliente in clientes_totais if cliente not in status_cliente_mix['nCdTerceiroCentralizador'].values],
    'total_compra_mix_cliente': 0,
    'cobertura_mix_clientes': 0,
    'produtos_unicos': [[]] * len([cliente for cliente in clientes_totais if cliente not in status_cliente_mix['nCdTerceiroCentralizador'].values]),
    'SKUs_Comprados_clientes': 0  # Adicionar quantidade como 0
})

# Combinar clientes com e sem mix
status_cliente_mix = pd.concat([status_cliente_mix, clientes_sem_mix], ignore_index=True)

# Explodir os produtos únicos em linhas separadas
clientes_comprando_mixIdeal = status_cliente_mix.explode('produtos_unicos')

# Renomear a coluna 'produtos_unicos' para algo mais intuitivo
clientes_comprando_mixIdeal.rename(columns={'produtos_unicos': 'nCdProduto_mix_clientes'}, inplace=True)

# Substituir NaN na coluna 'nCdProduto_mix_clientes' por 0 (caso de clientes sem produtos no mixIdeal)
clientes_comprando_mixIdeal['nCdProduto_mix_clientes'] = clientes_comprando_mixIdeal['nCdProduto_mix_clientes'].fillna(0).astype(int)

# Ordenar pelo maior valor de 'cobertura_mix_clientes'
clientes_comprando_mixIdeal = clientes_comprando_mixIdeal.sort_values(by=['cobertura_mix_clientes', 'nCdTerceiroCentralizador'], ascending=[False, True])
clientes_comprando_mixIdeal['nCdProduto_mix_clientes'] = clientes_comprando_mixIdeal['nCdProduto_mix_clientes'].astype(str)

# Exibir o DataFrame final
display(clientes_comprando_mixIdeal)


In [ ]:
# Agrupar pelos clientes e pela cobertura do mix, somando a quantidade de compras no mix
result = clientes_comprando_mixIdeal.groupby(['nCdTerceiroCentralizador', 'cobertura_mix_clientes'])['total_compra_mix_cliente'].sum().reset_index()

# Ordenar pelo maior valor de 'cobertura_mix_clientes'
result = result.sort_values(by='cobertura_mix_clientes', ascending=False)
display(result)
# Definir as faixas de porcentagem e os rótulos
bins = [-0.01, 0, 20, 40, 60, 80, 100]  # Adicionando o -0.01 para capturar os valores exatamente 0
labels = ['0%', '1%-20%', '21%-40%', '41%-60%', '61%-80%', '81%-100%']

# Classificar os clientes em faixas de porcentagem
result['percentage_range'] = pd.cut(result['cobertura_mix_clientes'], bins=bins, labels=labels, include_lowest=True)

# Contar o número de clientes em cada faixa
faixa_clientes_mix = result['percentage_range'].value_counts(sort=False).reset_index(name='num_clients')

# Renomear as colunas para maior clareza
faixa_clientes_mix.columns = ['Faixa de Porcentagem', 'Número de Clientes']

# Exibir o resultado
display(faixa_clientes_mix)

##(INSERIDO)3.Clientes que compraram produtos fora do mixIdeal (é possível pegar tbm os clientes que mais pegaram produtos fora do mixIdeal)


In [ ]:
# Filtrar os produtos fora do mixIdeal
dfForaMixIdeal = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='left', indicator=True)
dfForaMixIdeal = dfForaMixIdeal[dfForaMixIdeal['_merge'] == 'left_only']
dfForaMixIdeal.drop(columns=['_merge'], inplace=True)
# Agrupar por cliente (nCdTerceiroCentralizador) e produto (nCdProduto)
clientes_comprando_fora_mixIdeal = dfForaMixIdeal.groupby(['nCdTerceiroCentralizador', 'nCdProduto']).agg(
    qtds_produtos_comprados_fora_mixIdeal=('nCdProduto', 'size'),  # Contar a ocorrência de cada produto
    quantidade_caixas=('Quantidade de Caixas', 'sum')  # Somar a quantidade de caixas
).reset_index()

# Ordenar por 'counts' em ordem decrescente
clientes_comprando_fora_mixIdeal = clientes_comprando_fora_mixIdeal.sort_values(by='qtds_produtos_comprados_fora_mixIdeal', ascending=False)

# Exibir o DataFrame resultante
display(clientes_comprando_fora_mixIdeal)

##(INSERIDO )4.Coordenadores que venderam mais produtos do mixIdeal


In [ ]:
# Filtrar apenas os produtos do mixIdeal e agrupar por coordenador
dfComMixIdealCoordenador = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='inner')

# Agrupar para obter produtos únicos e a combinação de mix
mix = dfComMixIdealCoordenador[['nCdTerceiroCoordenador', 'nCdProduto']].groupby('nCdTerceiroCoordenador')['nCdProduto'].unique().reset_index(name='produtos_unicos')
mix['cobertura_mix'] = (mix['produtos_unicos'].apply(len) / len(mixIdeal['nCdProduto'])) * 100
mix['Mix Ideal únicos'] = mix['produtos_unicos'].apply(len)

# Adicionar a combinação_mix ao DataFrame original
dfComMixIdealCoordenador = pd.merge(dfComMixIdealCoordenador, mix[['nCdTerceiroCoordenador', 'cobertura_mix', 'Mix Ideal únicos']], on='nCdTerceiroCoordenador', how='left')

# Agrupar e calcular counts e soma da Quantidade de Caixas
coordenadores_vendendo_produtos_mix = dfComMixIdealCoordenador.groupby(['nCdTerceiroCoordenador', 'cobertura_mix', 'Mix Ideal únicos', 'nCdProduto']).agg(
    qtd_vendas_por_produtos=('nCdProduto', 'size'),  # Contagem de vendas por produto
    quantidade_caixas=('Quantidade de Caixas', 'sum')  # Soma da Quantidade de Caixas
).reset_index()



# Ordenar pelo cobertura_mix em ordem decrescente
coordenadores_vendendo_produtos_mix = coordenadores_vendendo_produtos_mix.sort_values(by='cobertura_mix', ascending=False)

# Exibir o DataFrame resultante
display(coordenadores_vendendo_produtos_mix)


In [ ]:
# Obter a lista completa de coordenadores
coordenadores_totais = notaVenda['nCdTerceiroCoordenador'].unique()

# Identificar coordenadores que não venderam nenhum produto do mix ideal
coordenadores_sem_mix = [
    coord for coord in coordenadores_totais if coord not in dfComMixIdealCoordenador['nCdTerceiroCoordenador'].unique()
]

# Criar DataFrame para coordenadores sem vendas no mix ideal
dfCoordenadoresSemMix = pd.DataFrame({
    'nCdTerceiroCoordenador': coordenadores_sem_mix,
    'cobertura_mix': 0,
    'Mix Ideal únicos': 0,
    'qtd_vendas_por_produtos': 0,
    'quantidade_caixas': 0
})

# Concatenar os dois DataFrames: com e sem vendas no mix ideal
coordenadores_mix_geral = pd.concat([coordenadores_vendendo_produtos_mix, dfCoordenadoresSemMix], ignore_index=True)

# Ordenar pelo valor de cobertura_mix
coordenadores_mix_geral = coordenadores_mix_geral.sort_values(by='cobertura_mix', ascending=False)

# Exibir o resultado final
coordenadores_mix_geral=coordenadores_mix_geral.groupby('nCdTerceiroCoordenador').first().reset_index().sort_values(by='cobertura_mix', ascending=False)

In [ ]:
# Criar as faixas de porcentagem e os rótulos
bins = [-0.01, 0, 20, 40, 60, 80, 100]  # Incluindo -0.01 para capturar valores exatamente 0
labels = ['0%', '1%-20%', '21%-40%', '41%-60%', '61%-80%', '81%-100%']

# Adicionar a coluna de faixa de porcentagem ao DataFrame de coordenadores
coordenadores_mix_geral['Faixa de Cobertura'] = pd.cut(
    coordenadores_mix_geral['cobertura_mix'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Selecionar apenas as colunas desejadas
resultado_final = coordenadores_mix_geral[[
    'nCdTerceiroCoordenador', 'cobertura_mix', 'qtd_vendas_por_produtos', 'Faixa de Cobertura'
]]

# Contar o número de coordenadores em cada faixa de cobertura
faixa_coordenadores = resultado_final['Faixa de Cobertura'].value_counts(sort=False).reset_index(name='Numero_Coordenadores')

# Renomear as colunas para maior clareza
faixa_coordenadores.columns = ['Faixa_Porcentagem', 'Numero_Coordenadores']
display(faixa_coordenadores)  # Contagem de coordenadores por faixa

##5.REDUNDANTE Quais coordenadores que, na venda total, negociaram mais produtos do mix ideal?

In [ ]:
# Filtrar vendas apenas com produtos no mix ideal
vendas_mix_ideal = notaVenda[notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Contar produtos únicos vendidos por cada coordenador
coordenadores_mix = vendas_mix_ideal.groupby('nCdTerceiroCoordenador')['nCdProduto'].unique().reset_index(name='produtos_unicos')

# Calcular a porcentagem de produtos do mix ideal (cobertura_mix)
coordenadores_mix['cobertura_mix'] = (coordenadores_mix['produtos_unicos'].apply(len) / len(mixIdeal['nCdProduto'])) * 100

# Contar o total de vendas do mix ideal por coordenador e somar a Quantidade de Caixas
vendas_coordenadores_mixIdeal = vendas_mix_ideal.groupby('nCdTerceiroCoordenador').agg(
    total_vendas_mix=('nCdProduto', 'count'),  # Total de vendas no mix ideal
    quantidade_caixas=('Quantidade de Caixas', 'sum')  # Soma da Quantidade de Caixas
).reset_index()

# Unificar as informações de coordenadores
vendas_coordenadores_mixIdeal = pd.merge(
    vendas_coordenadores_mixIdeal,
    coordenadores_mix[['nCdTerceiroCoordenador', 'cobertura_mix']],
    on='nCdTerceiroCoordenador',
    how='left'
)

# Criar a nova coluna que multiplica total_vendas_mix por quantidade_caixas
vendas_coordenadores_mixIdeal['multiplicacao_vendas_caixas'] = vendas_coordenadores_mixIdeal['total_vendas_mix'] * vendas_coordenadores_mixIdeal['quantidade_caixas']

# Ordenar pelo maior valor de 'cobertura_mix'
vendas_coordenadores_mixIdeal = vendas_coordenadores_mixIdeal.sort_values(by='cobertura_mix', ascending=False)

# Reorganizar as colunas para exibição
vendas_coordenadores_mixIdeal = vendas_coordenadores_mixIdeal[['nCdTerceiroCoordenador', 'cobertura_mix', 'total_vendas_mix', 'quantidade_caixas', 'multiplicacao_vendas_caixas']]

# Exibir o resultado
display(vendas_coordenadores_mixIdeal)

In [ ]:
# Agrupar pelos clientes e pela cobertura do mix, somando a quantidade de compras no mix
result = vendas_coordenadores_mixIdeal.groupby(['nCdTerceiroCoordenador', 'cobertura_mix'])['total_vendas_mix'].sum().reset_index()

# Ordenar pelo maior valor de 'cobertura_mix_clientes'
result = result.sort_values(by='cobertura_mix_clientes', ascending=False)
display(result)
# Definir as faixas de porcentagem e os rótulos
bins = [-0.01, 0, 20, 40, 60, 80, 100]  # Adicionando o -0.01 para capturar os valores exatamente 0
labels = ['0%', '1% - 20%', '21% - 40%', '41% - 60%', '61% - 80%', '81% - 100%']

# Classificar os clientes em faixas de porcentagem
result['percentage_range'] = pd.cut(result['cobertura_mix_clientes'], bins=bins, labels=labels, include_lowest=True)

# Contar o número de clientes em cada faixa
faixa_clientes_mix = result['percentage_range'].value_counts(sort=False).reset_index(name='num_clients')

# Renomear as colunas para maior clareza
faixa_clientes_mix.columns = ['Faixa de Porcentagem', 'Número de Clientes']

# Exibir o resultado
display(faixa_clientes_mix)

##(INSERIDO)6.Quais coordenadores estão atendendo o mix ideal nas vendas?


In [ ]:
# Calcular o total de SKUs ideais no mix ideal
total_skus_ideais = mixIdeal['nCdProduto'].nunique()

# Mesclar as vendas com mix ideal e coordenadores
vendas_com_mix = notaVenda.merge(mixIdeal, on="nCdProduto", how="inner")  # Apenas produtos do mix ideal
vendas_com_mix = vendas_com_mix.merge(coordenador, on="nCdTerceiroCoordenador", how="left")  # Adiciona coordenador

# Agrupar por coordenador para contar quantos SKUs do mix ideal ele vendeu
coordenadores_atendendo_mixIdeal = vendas_com_mix.groupby(["nCdTerceiroCoordenador", "cNmTerceiroCoordenador"]).agg(
    SKUs_Vendidos_coordenadores=("nCdProduto", "nunique"),
    Faturamento_Total_coordenadores=("FatValor", "sum")  # Soma do faturamento total
).reset_index()

coordenadores_atendendo_mixIdeal["cobertura_mix_coordenadores"] = (coordenadores_atendendo_mixIdeal["SKUs_Vendidos_coordenadores"] / total_skus_ideais) * 100


coordenadores_atendendo_mixIdeal = coordenadores_atendendo_mixIdeal.sort_values(by="cobertura_mix_coordenadores", ascending=False)
display(coordenadores_atendendo_mixIdeal)

##(INSERIDO)7.Calcular a razão (percentual de produtos vendidos no mixIdeal por total_vendas para cada coordenador)

In [ ]:
# Calcular o total de vendas por coordenador
total_vendas = notaVenda.groupby('nCdTerceiroCoordenador').agg(
    total_vendas=('nCdProduto', 'count'),
    quantidade_caixas=('Quantidade de Caixas', 'sum')  # Soma da Quantidade de Caixas
).reset_index()

# Filtrar apenas os produtos que estão no mixIdeal
vendas_produtos_no_mix = notaVenda[notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Obter o total de produtos vendidos no mixIdeal por coordenador
vendas_produtos_no_mix_total = vendas_produtos_no_mix.groupby('nCdTerceiroCoordenador').agg(
    vendas_produtos_no_mix=('nCdProduto', 'count'),
    caixas_no_mix=('Quantidade de Caixas', 'sum')  # Soma da Quantidade de Caixas no mixIdeal
).reset_index()

# Mesclar os totais com os produtos no mixIdeal para calcular a razão
coordenadores_razao_vendasMix_totalVendas = pd.merge(total_vendas, vendas_produtos_no_mix_total, on='nCdTerceiroCoordenador', how='left')

# Substituir valores NaN em 'vendas_produtos_no_mix' e 'caixas_no_mix' (caso um coordenador não tenha produtos no mixIdeal)
coordenadores_razao_vendasMix_totalVendas[['vendas_produtos_no_mix', 'caixas_no_mix']] = coordenadores_razao_vendasMix_totalVendas[['vendas_produtos_no_mix', 'caixas_no_mix']].fillna(0)

# Calcular a razão (percentual de produtos vendidos no mixIdeal por coordenador)
coordenadores_razao_vendasMix_totalVendas['razao_mix'] = (coordenadores_razao_vendasMix_totalVendas['vendas_produtos_no_mix'] / coordenadores_razao_vendasMix_totalVendas['total_vendas']) * 100

# Exibir o resultado ordenado pela maior razão
coordenadores_razao_vendasMix_totalVendas = coordenadores_razao_vendasMix_totalVendas.sort_values(by='razao_mix', ascending=False)

# Reorganizar as colunas para exibição
coordenadores_razao_vendasMix_totalVendas = coordenadores_razao_vendasMix_totalVendas[['nCdTerceiroCoordenador', 'razao_mix', 'total_vendas', 'vendas_produtos_no_mix', 'quantidade_caixas', 'caixas_no_mix']]

display(coordenadores_razao_vendasMix_totalVendas)

##(INSERIDO)8.Produtos do mixIdeal que mais venderam para cliente

In [ ]:
# Produtos do mixIdeal que mais venderam para cliente
# Filtrar apenas os produtos que estão no mixIdeal
produtos_mix_ideal = notaVenda[notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Contar a quantidade de clientes únicos (nCdTerceiroCentralizador) que compraram cada produto
produtosMix_comprados_clientes = produtos_mix_ideal.groupby('nCdProduto')['nCdTerceiroCentralizador'].nunique().reset_index(name='Quantidade de clientes')

# Ordenar os produtos do mixIdeal pela quantidade de clientes únicos em ordem decrescente
produtosMix_comprados_clientes = produtosMix_comprados_clientes.sort_values(by='Quantidade de clientes', ascending=False)

# Exibir o resultado
display(produtosMix_comprados_clientes)

##(INSERIDO)9.Produtos fora do mix Ideal que mais venderam para cliente



In [ ]:
#Produtos fora do mixIdeal que mais venderam para cliente
# Filtrar apenas os produtos que não estão no mixIdeal
produtos_fora_mix_ideal = notaVenda[~notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Contar a quantidade de clientes únicos (nCdTerceiroCentralizador) que compraram cada produto
produtosForaDOMix_comprados_clientes = produtos_fora_mix_ideal.groupby('nCdProduto')['nCdTerceiroCentralizador'].nunique().reset_index(name='Quantidade de clientes')

produtosForaDOMix_comprados_clientes = produtosForaDOMix_comprados_clientes.sort_values(by='Quantidade de clientes', ascending=False)
# Ordenar os produtos do mixIdeal pela quantidade de clientes únicos em ordem decrescente
display(produtosForaDOMix_comprados_clientes)

##(INSERIDO)10.Localidade dos clientes que mais compraram agrupado por estado dentro do Mix Ideal

In [ ]:
#notaVenda e cliente
notaVenda_com_uf = pd.merge(notaVenda, cliente[['nCdTerceiroCentralizador', 'UF']], on='nCdTerceiroCentralizador', how='left')
#  coluna AtendeMix
notaVenda_com_uf['AtendeMix'] = notaVenda_com_uf['nCdProduto'].isin(mixIdeal['nCdProduto'])

#atende MixIdeal
atende_mixIdeal = notaVenda_com_uf[notaVenda_com_uf['AtendeMix']]

localidades_compram_mixIdeal = atende_mixIdeal.groupby('UF').agg(
    Total_Fat_Cxs=('Fat Cxs', 'sum'),
    Total_Fat_R=('FatValor', 'sum'),
    Quantidade_Caixas=('Quantidade de Caixas', 'sum'),
    Frequencia=('UF', 'count')
).reset_index()
# Ordenar resultados pelo maior faturamento
localidades_compram_mixIdeal = localidades_compram_mixIdeal.sort_values(by='Total_Fat_R', ascending=False)

# Exibir os resultados
display(localidades_compram_mixIdeal)

##(INSERIDO)11.Localidade dos clientes que mais compraram por estado fora do Mix Ideal

In [ ]:
# n atende mix
nao_atende_mixIdeal = notaVenda_com_uf[~notaVenda_com_uf['AtendeMix']]
# Agrupar por UF e calcular os totais (Não Atende MixIdeal)
localidades_compram_fora_mixIdeal = nao_atende_mixIdeal.groupby('UF').agg(
    Total_Fat_Cxs=('Fat Cxs', 'sum'),
    Total_Fat_R=('FatValor', 'sum'),
    Quantidade_Caixas=('Quantidade de Caixas', 'sum'),
    Frequencia=('UF', 'count')
).reset_index()

localidades_compram_fora_mixIdeal = localidades_compram_fora_mixIdeal.sort_values(by='Total_Fat_R', ascending=False)
display(localidades_compram_fora_mixIdeal)

##(INSERIDO)12.Coordenadores que mais venderam do mixIdeal (por UF) (percentual por corbetura)

In [ ]:
# Merge dos dados com o DataFrame 'cliente' para trazer a localidade (UF)
notaVenda_com_localidade = pd.merge(
    notaVenda,
    cliente[['nCdTerceiroCentralizador', 'UF']],
    on='nCdTerceiroCentralizador',
    how='left'
)

# Identificar os produtos que pertencem ao MixIdeal
notaVenda_com_localidade['AtendeMix'] = notaVenda_com_localidade['nCdProduto'].isin(mixIdeal['nCdProduto'])

# Filtrar os produtos do MixIdeal
atende_mixIdeal = notaVenda_com_localidade[notaVenda_com_localidade['AtendeMix']]


coordenadores_vendendo_mixIdeal_por_UF = atende_mixIdeal.groupby(['nCdTerceiroCoordenador', 'UF']).agg(
    Quantidade_Vendas=('UF', 'count'),   # Contar as vendas baseadas no estado (UF)
    Faturamento_Total=('FatValor', 'sum')  # Soma do faturamento total
).reset_index()
coordenadores_vendendo_mixIdeal_por_UF = coordenadores_vendendo_mixIdeal_por_UF.rename(columns={'UF': 'UF_Cliente'})
coordenadores_vendendo_mixIdeal_por_UF = coordenadores_vendendo_mixIdeal_por_UF.sort_values(
    by=['nCdTerceiroCoordenador', 'Faturamento_Total'],
    ascending=[True, False]
)

display(coordenadores_vendendo_mixIdeal_por_UF)

##(INSERIDO)13.Coordenadores que mais venderam fora do mixIdeal por UF

In [ ]:
#Merge dos dados com o DataFrame 'cliente' para trazer a localidade (UF)
notaVenda_com_localidade = pd.merge(
    notaVenda,
    cliente[['nCdTerceiroCentralizador', 'UF']],
    on='nCdTerceiroCentralizador',
    how='left'
)
# coluna "AtendeMix" para identificar se o produto está no MixIdeal
notaVenda_com_localidade['AtendeMix'] = notaVenda_com_localidade['nCdProduto'].isin(mixIdeal['nCdProduto'])

#  filtrar os produtos que NÃO estão no MixIdeal
nao_atende_mixIdeal = notaVenda_com_localidade[~notaVenda_com_localidade['AtendeMix']]
coordenadores_vendendo_fora_mixIdeal_por_UF = nao_atende_mixIdeal.groupby(['nCdTerceiroCoordenador', 'UF']).agg(
    Quantidade_Vendas=('UF', 'count'),   # Contar as vendas baseadas no estado (UF)
    Faturamento_Total=('FatValor', 'sum')  # Soma do faturamento total
).reset_index()

coordenadores_vendendo_fora_mixIdeal_por_UF = coordenadores_vendendo_fora_mixIdeal_por_UF.sort_values(
    by=['nCdTerceiroCoordenador', 'Faturamento_Total'],
    ascending=[True, False]
)

display(coordenadores_vendendo_fora_mixIdeal_por_UF)

##(INSERIDO)14.Análise do faturamento por produto, comparando as vendas de produtos que estão dentro do mix ideal com aqueles que estão fora do mix ideal

In [ ]:
group_fats_prod = notaVenda.groupby("nCdProduto").agg(
    Qtd_vendidos=("nCdProduto", "size"),
    Fat_Cxs=("Fat Cxs", "sum"),
    Fat_R=("FatValor", "sum"),
    Quantidade_Caixas=('Quantidade de Caixas', 'sum')
).reset_index()

faturamentos_por_produtos_mixIdeal = group_fats_prod[group_fats_prod["nCdProduto"].isin(mixIdeal["nCdProduto"])]
display(len(faturamentos_por_produtos_mixIdeal))
display(faturamentos_por_produtos_mixIdeal)

faturamentos_por_produtos_fora_mixIdeal = group_fats_prod[~group_fats_prod["nCdProduto"].isin(mixIdeal["nCdProduto"])]
display(len(faturamentos_por_produtos_fora_mixIdeal))
display(faturamentos_por_produtos_fora_mixIdeal)

calculate_totals = lambda df: {
    "Total Fat Cxs": df["Fat_Cxs"].sum(),
    "Total Fat R$": df["Fat_R"].sum(),
}
totals_naoatende = calculate_totals(faturamentos_por_produtos_fora_mixIdeal)
totals_atende = calculate_totals(faturamentos_por_produtos_mixIdeal)

for category, totals in [("Não atende MixIdeal", totals_naoatende), ("Atende MixIdeal", totals_atende)]:
    print(f"Resultados para {category}:")
    for metric, total in totals.items():
        print(f"  {metric}: {total}")

##15(INSERIDO).Há alguma relação entre o desempenho de coordenadores e a cobertura do mix ideal de seus clientes?

In [ ]:
# Passo 1: Total de vendas por coordenador e cliente
total_vendas_coordenador_cliente = notaVenda.groupby(['nCdTerceiroCoordenador', 'nCdTerceiroCentralizador']).agg(
    total_faturamento=('FatValor', 'sum'),  # Total de faturamento
    total_vendas=('nCdProduto', 'count')  # Total de produtos vendidos
).reset_index()

# Passo 2: Filtrar os produtos do Mix Ideal
vendas_mix_ideal = notaVenda[notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Passo 3: Calcular o total de produtos do Mix Ideal negociados por coordenador e cliente
produtos_mix_totais = vendas_mix_ideal.groupby(['nCdTerceiroCoordenador', 'nCdTerceiroCentralizador'])['nCdProduto'] \
    .count() \
    .reset_index(name='total_vendas_mix')

# Passo 4: Mesclar com o total de vendas para cada coordenador e cliente
relacao_vendas_coordenador_cliente = pd.merge(
    total_vendas_coordenador_cliente,
    produtos_mix_totais,
    on=['nCdTerceiroCoordenador', 'nCdTerceiroCentralizador'],
    how='left'
)

# Preencher valores NaN em 'total_vendas_mix' para coordenadores/clientes sem produtos do Mix Ideal
relacao_vendas_coordenador_cliente['total_vendas_mix'] = relacao_vendas_coordenador_cliente['total_vendas_mix'].fillna(0).astype(int)

# Passo 5: Calcular a relação do mix (total de produtos do Mix Ideal / total de produtos vendidos)
relacao_vendas_coordenador_cliente['relacao_mix'] = (
    relacao_vendas_coordenador_cliente['total_vendas_mix'] /
    relacao_vendas_coordenador_cliente['total_vendas']
) * 100

# Ordenar o resultado por coordenador e cliente
relacao_vendas_coordenador_cliente = relacao_vendas_coordenador_cliente.sort_values(
    by=['nCdTerceiroCoordenador', 'nCdTerceiroCentralizador']
)

relacao_vendas_coordenador_cliente = relacao_vendas_coordenador_cliente.sort_values(by='total_vendas', ascending=False)

# Exibir o DataFrame final
display(relacao_vendas_coordenador_cliente)

##16.Quais SKUs apresentam maiores lacunas de venda em relação ao mix ideal? (os que não compram nada do mix ideal, gráfico de barras - fazer uma correlação dos produtos que eles compram) - Que estratégias podem ser implementas para melhorar a cobertura em clientes específicos?

In [ ]:
# Utilizar QLik para responder

##(INSERIDO)17.Qual negócio está faturando mais?

In [ ]:
#  Negócio e somar o faturamento
faturamento_por_negocio = notaVenda.groupby('Negócio')['FatValor'].sum().reset_index()
faturamento_por_negocio = faturamento_por_negocio.sort_values(by='FatValor', ascending=False)

faturamento_por_negocio = faturamento_por_negocio.rename(columns={'Negócio': 'Nome do Negócio', 'FatValor': 'Faturamento'})

# Exibir o top 10 negócios que mais compraram
display(faturamento_por_negocio)


##(INSERIDO)18.Empresas que estão comprando o mix ideal

In [ ]:
# Filtrar as linhas que representam compras do mix ideal com base no 'ncdProduto'

# Merge 'notaVenda' and 'empresa' DataFrames to include company information.
notaVenda_empresa = pd.merge(notaVenda, empresa, on='nCdEmpresa', how='left')

# Filter sales data for products in the ideal mix using 'nCdProduto' from 'mixIdeal'.
mix_ideal_produtos = mixIdeal['nCdProduto'].tolist()  # Get all 'nCdProduto' from 'mixIdeal'
mix_ideal_vendas = notaVenda_empresa[notaVenda_empresa['nCdProduto'].isin(mix_ideal_produtos)]

# Count sales by company for products in the ideal mix and get the top 5 companies.
empresas_comprando_mixIdeal= mix_ideal_vendas['Empresa'].value_counts().sort_values(ascending=False).reset_index()

display(empresas_comprando_mixIdeal)

##19.Quantas vendas são feitas entre coordenadores e clientes dos produtos do mixIdeal

In [ ]:
# Quantas vendas são feitas entre coordenadores e clientes dos produtos do mixIdeal
notaVenda['AtendeMix'] = notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])

resultado_grouped = (
    notaVenda
    .groupby(['nCdTerceiroCentralizador', 'nCdTerceiroCoordenador', 'AtendeMix'])
    .agg({
        'Negócio': 'count',  # Contagem de negócios
        'Fat Cxs': 'sum',
        'FatValor': 'sum'
    })

    .rename(columns={'Negócio': 'Frequencia'})
    .reset_index()
)
# Separar em dois DataFrames: AtendeMix e NaoAtendeMix
vendas_coordenadores_clientes_mixIdeal = resultado_grouped[resultado_grouped['AtendeMix'] == True].drop(columns='AtendeMix')
vendas_coordenadores_clientes_fora_mixIdeal = resultado_grouped[resultado_grouped['AtendeMix'] == False].drop(columns='AtendeMix')

# Renomear colunas para clareza em ambos os DataFrames
vendas_coordenadores_clientes_mixIdeal.rename(columns={
    'Fat Cxs': 'FatCxs_AtendeMix',
    'FatValor': 'FatValor_AtendeMix',
    'Frequencia': 'Frequencia_AtendeMix'
}, inplace=True)

vendas_coordenadores_clientes_fora_mixIdeal.rename(columns={
    'Fat Cxs': 'FatCxs_NaoAtendeMix',
    'FatValor': 'FatValor_NaoAtendeMix',
    'Frequencia': 'Frequencia_NaoAtendeMix'
}, inplace=True)

# Exibir DataFrames separados
print("DataFrame AtendeMix:")
display(vendas_coordenadores_clientes_mixIdeal)

print("\nDataFrame NaoAtendeMix:")
display(vendas_coordenadores_clientes_fora_mixIdeal)

##(INSERIDO)20.Qual a marca que mais faturou no total


In [ ]:
# Adicionar a coluna de marca associando com o DataFrame de produtos
vendas_com_marca = pd.merge(notaVenda, produto[['nCdProduto', 'Marca']], on='nCdProduto', how='inner')

# Calcular o total de faturamento e a quantidade de vendas por marca
vendas_por_marca = vendas_com_marca.groupby('Marca').agg(
    faturamento_total=('FatValor', 'sum'),  # Soma do faturamento
    quantidade_vendas=('nCdProduto', 'count')  # Contagem de vendas
).reset_index()

# Ordenar as marcas pelo faturamento total em ordem decrescente
vendas_por_marca = vendas_por_marca.sort_values(by='faturamento_total', ascending=False)

# Exibir o resultado
display(vendas_por_marca)


##(INSERIDO)21.Qual a marca que mais vendeu do mix ideal



In [ ]:
# Filtrar apenas os produtos do mixIdeal
vendas_mix_ideal = notaVenda[notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Adicionar a coluna de marca associando com o DataFrame de produtos
vendas_mix_ideal_com_marca = pd.merge(vendas_mix_ideal, produto[['nCdProduto', 'Marca']], on='nCdProduto', how='inner')

# Calcular o total de vendas e faturamento por marca (apenas para produtos do mix ideal)
vendas_por_marca_mix_ideal = vendas_mix_ideal_com_marca.groupby('Marca').agg(
    total_vendas=('nCdProduto', 'count'),   # Contagem de vendas por marca
    faturamento_total=('FatValor', 'sum')  # Soma do faturamento por marca
).reset_index()

# Ordenar as marcas pelo total de vendas em ordem decrescente
vendas_por_marca_mix_ideal = vendas_por_marca_mix_ideal.sort_values(by='total_vendas', ascending=False)

# Exibir o resultado
display(vendas_por_marca_mix_ideal)


##(INSERIDO)22.Qual a marca que mais vendeu fora do mix ideal


In [ ]:
# Filtrar apenas os produtos do mixIdeal
vendas_mix_ideal = notaVenda[~notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Adicionar a coluna de marca associando com o DataFrame de produtos
vendas_mix_ideal_com_marca = pd.merge(vendas_mix_ideal, produto[['nCdProduto', 'Marca']], on='nCdProduto', how='inner')

# Calcular o total de vendas por marca (apenas para produtos do mix ideal)
vendas_por_marca_fora_mixIdeal = vendas_mix_ideal_com_marca.groupby('Marca').agg(
    total_vendas=('nCdProduto', 'count'),   # Contagem de vendas por marca
    faturamento_total=('FatValor', 'sum')  # Soma do faturamento por marca
).reset_index()

# Ordenar as marcas pelo total de vendas em ordem decrescente
vendas_por_marca_fora_mixIdeal = vendas_por_marca_fora_mixIdeal.sort_values(by='total_vendas', ascending=False)

# Exibir todas as marcas do mix ideal ordenadas por vendas
display(vendas_por_marca_fora_mixIdeal)

##(INSERIDO)23.Qual a categoria que mais faturou no mix ideal



In [ ]:
#filtra os produtos que estão no mix ideal
produtos_no_mix_ideal = notaVenda[notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# Adicionar a categoria associando com o mixIdeal
produtos_com_categoria = pd.merge(produtos_no_mix_ideal, mixIdeal[['nCdProduto', 'Categoria']], on='nCdProduto', how='left')

# Agrupar por categoria e calcular total de faturamento e quantidade de vendas
produtos_com_categoria = produtos_com_categoria.groupby('Categoria').agg(
    total_faturamento=('FatValor', 'sum'),   # Soma do faturamento por categoria
    total_vendas=('nCdProduto', 'count')    # Contagem de vendas por categoria
).reset_index()

# Ordenar pela maior soma de faturamento
categoria_mais_faturou_mixIdeal = produtos_com_categoria.sort_values(by='total_faturamento', ascending=False)

# Exibir o resultado
display(categoria_mais_faturou_mixIdeal)

##(INSERIDO)24.Qual a categoria que mais faturou fora do mix ideal

In [ ]:
notaVenda_com_produto = pd.merge(
    notaVenda,
    produto[['nCdProduto', 'Categoria', 'Classificação', 'Tipo', 'Produto']],
    on='nCdProduto',
    how='left'
)

# Coluna "AtendeMix" para identificar se o produto está no MixIdeal
notaVenda_com_produto['AtendeMix'] = notaVenda_com_produto['nCdProduto'].isin(mixIdeal['nCdProduto'])

# Filtrar os produtos que NÃO estão no MixIdeal
nao_atende_mixIdeal = notaVenda_com_produto[~notaVenda_com_produto['AtendeMix']]

# Agrupar por categorias e calcular as métricas
categoria_mais_faturou_fora_mixIdeal = nao_atende_mixIdeal.groupby('Categoria').agg(
    Frequencia=('Categoria', 'count'),          # Contagem de frequência
    Total_Fat_Cxs=('Fat Cxs', 'sum'),           # Soma do faturamento em caixas
    Total_Fat_R=('FatValor', 'sum'),            # Soma do faturamento total
    Total_Vendas=('nCdProduto', 'count')        # Total de vendas (número de produtos vendidos)
).reset_index()

# Calcular a razão de Total_Fat_Cxs por Total_Fat_R (Caixas por Faturamento)
categoria_mais_faturou_fora_mixIdeal['Quantidade de caixas'] = categoria_mais_faturou_fora_mixIdeal['Total_Fat_Cxs'] / categoria_mais_faturou_fora_mixIdeal['Total_Fat_R']

# Ordenar o DataFrame pelo maior faturamento
categoria_mais_faturou_fora_mixIdeal = categoria_mais_faturou_fora_mixIdeal.sort_values(by='Total_Fat_R', ascending=False)

# Exibir o DataFrame final
display(categoria_mais_faturou_fora_mixIdeal)


#Solução de problemas

In [ ]:
''' Solução do problema: Qual mudança precisamos fazer pra melhorar os resutados?
                         Qual a porcentagem que temos no modelo atual, e qual a porcentagem que precisamos melhorar?
                         Precisamos concentrar as vendas em qual coordenador?
                         Precisamos focar em qual cliente?
                         precisamos focar em qual produto?
                         precisamos focar em qual divisão comercial?
                         precisamos focar em qual estado?
                         precisamos focar em qual mês?
'''

#Enviar DFs para a BigQuery

In [ ]:
for dataframe in \
 [mixIdeal,
  empresa,
  produto,
  divisaoComercial,
  cliente,
  coordenador,
  notaVenda,
  produtos_mix_ideal,
  produtos_fora_mix_ideal,
  clientes_comprando_fora_mixIdeal,
  produtosForaDOMix_comprados_clientes,
  clientes_comprando_mixIdeal,
  coordenadores_atendendo_mixIdeal,
  clientes_qtds_mixIdeal,
  coordenadores_vendendo_produtos_mix,
  coordenadores_razao_vendasMix_totalVendas,
  produtosMix_comprados_clientes,
  localidades_compram_mixIdeal,
  faturamentos_por_produtos_mixIdeal,
  faturamentos_por_produtos_fora_mixIdeal,
  relacao_vendas_coordenador_cliente,
  faturamento_por_negocio,
  empresas_comprando_mixIdeal,
  vendas_por_marca,
  vendas_por_marca_mix_ideal,
  vendas_por_marca_fora_mixIdeal,
  categoria_mais_faturou_mixIdeal,
  categoria_mais_faturou_fora_mixIdeal,
  localidades_compram_fora_mixIdeal,
  coordenadores_vendendo_mixIdeal_por_UF,
  coordenadores_vendendo_fora_mixIdeal_por_UF,
  faixa_clientes_mix,
  faixa_coordenadores]:
  enviar_para_bigquery(faixa_coordenadores)

In [ ]:
# Calcular o total de SKUs ideais no mix ideal
total_skus_ideais = mixIdeal['nCdProduto'].nunique()

# Mesclar as vendas com mix ideal e coordenadores
vendas_com_mix = notaVenda.merge(mixIdeal, on="nCdProduto", how="inner")  # Apenas produtos do mix ideal
vendas_com_mix = vendas_com_mix.merge(coordenador, on="nCdTerceiroCoordenador", how="left")  # Adiciona coordenador

# Agrupar por coordenador para contar quantos SKUs do mix ideal ele vendeu
coordenadores_atendendo_mixIdeal = vendas_com_mix.groupby(["nCdTerceiroCoordenador", "cNmTerceiroCoordenador"]).agg(
    SKUs_Vendidos_coordenadores=("nCdProduto", "nunique"),
    Faturamento_Total_coordenadores=("FatValor", "sum")  # Soma do faturamento total
).reset_index()

coordenadores_atendendo_mixIdeal["cobertura_mix_coordenadores"] = (coordenadores_atendendo_mixIdeal["SKUs_Vendidos_coordenadores"] / total_skus_ideais) * 100


coordenadores_atendendo_mixIdeal = coordenadores_atendendo_mixIdeal.sort_values(by="cobertura_mix_coordenadores", ascending=False)
display(coordenadores_atendendo_mixIdeal)

# Análises Descritivas e Medidas

In [ ]:
# seleciona apenas as colunas quantitativas para realizar a correlação entre elas
correlacao = notaVenda[['FatValor','Fat Cxs', 'Quantidade de Caixas']].corr()
figura = plt.figure(figsize=(4, 4))
plt.title('Correlação entre o Valor do Faturamento (R$), Faturamento das Caixas e Quantidade de Caixas')
sns.heatmap(correlacao, annot=True, cmap='PuOr', vmin=-1, vmax=1);

In [ ]:
colunas_analisadas = ['FatValor','Fat Cxs', 'Quantidade de Caixas']
for coluna in colunas_analisadas:
        media = np.mean(notaVenda[coluna])
        mediana = np.median(notaVenda[coluna])
        moda = st.mode(notaVenda[coluna])
        variancia = st.variance(notaVenda[coluna])
        desviopadrao = st.stdev(notaVenda[coluna])
        valorminimo = np.min(notaVenda[coluna])
        quartis = st.quantiles(notaVenda[coluna], n=4)
        umquartil = quartis[0]
        doisquartil = quartis[1]
        tresquartil = quartis[2]
        valormaximo = np.max(notaVenda[coluna])
        amplitude = tresquartil - umquartil
        liminferior = max(umquartil - (1.5 * amplitude), valorminimo)
        limsuperior = max(tresquartil + (1.5 * amplitude), valormaximo)

        estatisticas = {
            'Métrica': [
                'Média', 'Mediana', 'Moda', 'Variância', 'Desvio Padrão',
                'Valor Mínimo', '1º Quartil', '2º Quartil (Mediana)', '3º Quartil',
                'Valor Máximo', 'Amplitude', 'Limite Inferior', 'Limite Superior'
            ],
            'Valor': [
                media, mediana, moda, variancia, desviopadrao,
                valorminimo, umquartil, doisquartil, tresquartil,
                valormaximo, amplitude, liminferior, limsuperior
            ]
        }

        # Criando um DataFrame com as estatísticas
        df_estatisticas = pd.DataFrame(estatisticas)

        # Exibindo os resultados
        print(f"Resultados para '{coluna}':")
        display(df_estatisticas)
        print('-----------------------------------------------')

In [ ]:
#produtos vendidos estão no mix Ideal
df_completo = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='inner')
print("\nProdutos Vendidos no Mix Ideal:")
display(df_completo)

#contando a quantidade de cada produto vendido que está no mix ideal
contagem_completo = df_completo.groupby('nCdProduto').size().reset_index(name='Quantidade Vendida')
print("\nContagem dos Produtos Vendidos no Mix Ideal:")
display(contagem_completo)

#produtos vendidos NÃO estão no mix ideal
naoestaonomix = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='left')
produtos_nao_no_mix = naoestaonomix[naoestaonomix['Marca'].isna()]
print("\nProdutos Vendidos que NÃO Estão no Mix Ideal:")
display(produtos_nao_no_mix)

#quantidade de cada produto vendido que não está no mix ideal
contagem_nao_no_mix = produtos_nao_no_mix.groupby('nCdProduto').size().reset_index(name='Quantidade')
print("\nContagem dos Produtos Vendidos que NÃO Estão no Mix Ideal:")
display(contagem_nao_no_mix)

#verificando se todos os produtos do Mix Ideal foram vendidos
todosmixvendidos = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='right')
produtos_no_mix_nao_vendidos = todosmixvendidos[todosmixvendidos['Fat Cxs'].isna()]
print("\nProdutos no Mix Ideal que NÃO Foram Vendidos:")
display(produtos_no_mix_nao_vendidos)

#contando os produtos do mix ideal que não foram vendidos
contagem_no_mix_nao_vendidos = produtos_no_mix_nao_vendidos.groupby('nCdProduto').size().reset_index(name='Quantidade Não Vendida')
print("\nContagem dos Produtos no Mix Ideal que NÃO Foram Vendidos:")
display(contagem_no_mix_nao_vendidos)


In [ ]:
# Contar o número de registros para cada 'nCdProduto'
contagem_produtos = notaVenda.groupby('nCdProduto').size().reset_index(name='Contagem')
# Ordenar pela quantidade de vendas em ordem decrescente
produtos_mais_vendidos = contagem_produtos.sort_values(by='Contagem', ascending=False).head(5)
#Renomeando as colunas
produtos_mais_vendidos = produtos_mais_vendidos.rename(columns={'nCdProduto': 'Código do Produto', 'Contagem': 'Contagem de Vendas'})
print(produtos_mais_vendidos.to_string(index=False))

In [ ]:
# mesclando as tabelas de vendas e mix ideal com base no código do produto
no = pd.merge(notaVenda, mixIdeal, on='nCdProduto', how='inner')

# Agrupando por código do produto e somando o fatValor
menos_vendidos = df_completo.groupby(['Marca', 'Produto'])['FatValor'].sum().reset_index()

# Deixando em crescente
menos_vendidos = menos_vendidos.sort_values(by='FatValor', ascending=True)

print("\nProdutos Menos Vendidos do Mix Ideal:")
display(menos_vendidos)

#SCHEMA SQL

In [ ]:
!pip install mysql-connector-python
import mysql.connector

In [ ]:
host = '104.198.179.63' # endereço do banco externo que eu quero acessar
user = 'root'
password = ''
database = 'Flora'

In [ ]:
'''
CREATE SCHEMA IF NOT EXISTS Flora;
USE Flora;


CREATE TABLE DimEmpresa (
    nCdEmpresa INT PRIMARY KEY,
    Empresa VARCHAR(255) NOT NULL
);

CREATE TABLE DimDivisaoComercial (
    nCdDivisaoComercial INT PRIMARY KEY,
    DivisaoComercial VARCHAR(255) NOT NULL
);

CREATE TABLE DimCoordenador (
    nCdTerceiroCoordenador INT PRIMARY KEY,
    cNmTerceiroCoordenador VARCHAR(255) NOT NULL
);

CREATE TABLE DimCliente (
    nCdTerceiroCentralizador INT PRIMARY KEY,
    TerceiroCentralizador VARCHAR(255) NOT NULL,
    UF CHAR(2) NOT NULL
);

CREATE TABLE DimProduto (
    nCdProduto INT PRIMARY KEY,
    Marca VARCHAR(255),
    Categoria VARCHAR(255),
    Classificacao VARCHAR(255),
    Tipo VARCHAR(255),
    Produto VARCHAR(255) NOT NULL
);

CREATE TABLE MixIdeal (
    nCdProduto INT PRIMARY KEY,
    Marca VARCHAR(255),
    Categoria VARCHAR(255),
    Produto VARCHAR(255),
    FOREIGN KEY (nCdProduto) REFERENCES DimProduto(nCdProduto)
);

CREATE TABLE FatoNotaVenda (
    Negocio VARCHAR(15),
    DataFaturamento DATE NOT NULL,
    Mes TINYINT,
    Ano INT,
    nCdEmpresa INT,
    nCdProduto INT,
    nCdDivisaoComercial INT,
    nCdTerceiroCoordenador INT,
    nCdTerceiroCentralizador INT,
    FatCxs FLOAT,
    FatR$ FLOAT,
    FOREIGN KEY (nCdEmpresa) REFERENCES DimEmpresa(nCdEmpresa),
    FOREIGN KEY (nCdProduto) REFERENCES DimProduto(nCdProduto),
    FOREIGN KEY (nCdDivisaoComercial) REFERENCES DimDivisaoComercial(nCdDivisaoComercial),
    FOREIGN KEY (nCdTerceiroCoordenador) REFERENCES DimCoordenador(nCdTerceiroCoordenador),
    FOREIGN KEY (nCdTerceiroCentralizador) REFERENCES DimCliente(nCdTerceiroCentralizador)
);

'''

In [ ]:
#produtos que estão no mix ideal, mas que o cliente não comprou
clientes_sem_mix_ideal_agrupados = notaVenda[~notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])] \
    .groupby('nCdTerceiroCentralizador')['nCdProduto'].nunique().reset_index()

#renomear a coluna de contagem
clientes_sem_mix_ideal_agrupados.columns = ['nCdTerceiroCentralizador', 'produtos_nao_comprados_mix_ideal']

#dataFrame de clientes que não compraram produtos do mix ideal
clientes_sem_mix_ideal_df = clientes_sem_mix_ideal_agrupados.reset_index()

#ordenar os clientes pela contagem de produtos não comprados do mix ideal
clientes_sem_mix_ideal_df = clientes_sem_mix_ideal_df.sort_values(by='produtos_nao_comprados_mix_ideal', ascending=False)

#merge para adicionar informações de coordenador, localidade e empresa
clientes_com_coordenador = pd.merge(clientes_sem_mix_ideal_df,
                                    notaVenda[['nCdTerceiroCentralizador', 'nCdTerceiroCoordenador', 'nCdDivisaoComercial', 'nCdEmpresa']].drop_duplicates(),
                                    on='nCdTerceiroCentralizador', how='left')

#merge com DimCoordenador para obter o nome do coordenador
clientes_com_coordenador = pd.merge(clientes_com_coordenador,
                                    coordenador[['nCdTerceiroCoordenador', 'cNmTerceiroCoordenador']],
                                    on='nCdTerceiroCoordenador', how='left')

#merge com divisaoComercial para obter o nome da divisão comercial
clientes_com_coordenador = pd.merge(clientes_com_coordenador,
                                    divisaoComercial[['nCdDivisaoComercial', 'DivisaoComercial']],
                                    on='nCdDivisaoComercial', how='left')

#merge com Empresa para adicionar o nome da empresa
clientes_com_coordenador = pd.merge(clientes_com_coordenador,
                                    empresa[['nCdEmpresa', 'Empresa']],
                                    on='nCdEmpresa', how='left')

#agrupar por empresa e somar a contagem de produtos não comprados
produtos_nao_comprados_por_empresa = clientes_com_coordenador.groupby('Empresa')['produtos_nao_comprados_mix_ideal'].sum().reset_index()
# exibir o DataFrame final com as informações detalhadas dos clientes
display(clientes_com_coordenador[['nCdTerceiroCentralizador', 'produtos_nao_comprados_mix_ideal', 'cNmTerceiroCoordenador']])

##Produtos que não estão no mix Ideal

In [ ]:
#filtrar os produtos que NÃO estão no mix ideal (comprados fora do mix ideal)
produtos_fora_mix = notaVenda[~notaVenda['nCdProduto'].isin(mixIdeal['nCdProduto'])]

# contar quantos produtos fora do mix ideal cada cliente comprou
clientes_com_produtos_fora_mix = produtos_fora_mix.groupby(['nCdTerceiroCentralizador', 'nCdTerceiroCoordenador'])['nCdProduto'].nunique().reset_index()

# renomear a coluna para melhor compreensão
clientes_com_produtos_fora_mix.columns = ['nCdTerceiroCentralizador', 'nCdTerceiroCoordenador', 'produtos_fora_mix']

# merge com coordenador para obter o nome do coordenador
clientes_com_coordenador = pd.merge(clientes_com_produtos_fora_mix,
                                    coordenador[['nCdTerceiroCoordenador', 'cNmTerceiroCoordenador']],
                                    on='nCdTerceiroCoordenador', how='left')

# agrupar por coordenador e somar a quantidade de produtos comprados fora do mix
produtos_fora_mix_por_coordenador = clientes_com_coordenador.groupby('cNmTerceiroCoordenador')['produtos_fora_mix'].sum().reset_index()

# ordenar por número de produtos fora do mix (decrescente)
produtos_fora_mix_por_coordenador = produtos_fora_mix_por_coordenador.sort_values(by='produtos_fora_mix', ascending=False)
display(produtos_fora_mix_por_coordenador)

# Sugestoes

In [ ]:
display(vendas_coordenadores_mixIdeal['cobertura_mix'].mean())

In [ ]:
display(vendas_coordenadores_mixIdeal['cobertura_mix'].median())

In [ ]:
display(coordenadores_atendendo_mixIdeal['Faturamento_Total_coordenadores'].sum())

In [ ]:
display(coordenadores_razao_vendasMix_totalVendas['vendas_produtos_no_mix'].sum())

In [ ]:
display(coordenadores_razao_vendasMix_totalVendas['vendas_produtos_no_mix'].mean())